In [ ]:
import psycopg2
import pandas as pd

### Create connection to database

In [ ]:
try:
    conn = psycopg2.connect("host=127.0.0.1 dbname=hockey_YT_Project user=postgres password=password")
except psycopg2.Error as e:
    print("Error: Could not connect to database")
    print(e)

try:
    cur = conn.cursor()
except psycopg2.Error as e:
    print("Error: Could not get cursor to the database")
    print(e)

conn.set_session(autocommit=True)

#### create table for dim_dates

In [ ]:
try:
    cur.execute("CREATE TABLE IF NOT EXISTS dim_dates (\
                id int,\
                date date,\
                day varchar,\
                week int,\
                is_weekend int,\
                is_holiday int);")
    
except psycopg2.Error as e:
    print("Error: Could not create table")
    print(e)

#### Insert values into table dim_dates

In [ ]:
## load DF for dates
df_dates = pd.read_csv("/Users/mattrende/Documents/Coding/hockey-reference/YouTube Project/csv/out/dim_dates.csv", parse_dates=["Date"])
df_dates.head(2)

In [ ]:
date_insert = ("""INSERT INTO dim_dates(
               id,
               date,
               day,
               week,
               is_weekend,
               is_holiday)
               VALUES (%s, %s, %s, %s, %s, %s)
               """)

In [ ]:
try:
    for i, row in df_dates.iterrows():
        #print(list(row))
        cur.execute(date_insert, list(row))

except psycopg2.Error as e:
    print("Error: could not insert rows")
    print(e)

In [ ]:
try: 
    cur.execute("SELECT * From dim_dates;")

except psycopg2.Error as e:
    print("Select Error")
    print(e)

row = cur.fetchone()
while row:
    print(row)
    row = cur.fetchone()

### Close cursor and connection

In [ ]:
cur.close()
conn.close()